# Leto Demo Notebook

## Install all requirements

In [2]:
! pwd

/root/Github/aEye/runner_notebooks


In [3]:
cd ..

/root/Github/aEye


In [4]:
! pwd

/root/Github/aEye


In [5]:
!pip install -r requirements.txt

In [6]:
!apt-get update && apt-get install libgl1 -y

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://security.debian.org/debian-security buster/updates InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1 is already the newest version (1.1.0-1).
0 upgraded, 0 newly installed, 0 to remove and 67 not upgraded.


move to root of repo to run code. 

In [26]:
! pip install opencv-contrib-python

  Using cached opencv_contrib_python-4.8.0.74-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (67.8 MB)


In [27]:
! pip install --upgrade opencv-python

In [7]:
! pwd

/root/Github/aEye


## Imports

In [5]:
import boto3
import cv2
from cv2 import dnn_superres
import logging
from aEye.video import Video
from aEye.processor import Processor
from aEye.auxiliary import Aux
root = logging.getLogger()
root.setLevel(logging.INFO)
root.setLevel(logging.DEBUG)
root.setLevel(logging.ERROR)

## Create objects to process original video into the correct sizing

In [6]:
aux = Aux()

In [7]:
process = Processor()

# Section 1: Trimming original video to 10 seconds. 

In [8]:
video_list_s3 = aux.load_s3(bucket = 'leto-dish', prefix = 'original-videos/random-videos/')

In [9]:
#print the metadata
print(video_list_s3[0].get_meta_data())

{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'High', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 1920, 'height': 1080, 'coded_width': 1920, 'coded_height': 1080, 'closed_captions': 0, 'film_grain': 0, 'has_b_frames': 2, 'pix_fmt': 'yuv420p', 'level': 40, 'color_range': 'tv', 'color_space': 'bt709', 'color_transfer': 'bt709', 'color_primaries': 'bt709', 'chroma_location': 'left', 'field_order': 'progressive', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'id': '0x1', 'r_frame_rate': '25/1', 'avg_frame_rate': '25/1', 'time_base': '1/12800', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 128000, 'duration': '10.000000', 'bit_rate': '79042', 'bits_per_raw_sample': '8', 'nb_frames': '250', 'extradata_size': 50, 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impaired': 0, 'clea

From the metadata we can see that the original codec is h.264, and the video is 1080p

### Turn video into 10 second clip -- only performed once. 

In [10]:
# trimmed_s3 = process.add_label_trimming_start_duration(video_list_s3,0,10)

In [11]:
# aux.execute_label_and_write_local(trimmed_s3)

In [12]:
# aux.upload_s3(trimmed_s3, bucket = 'leto-dish', prefix = 'original-videos/random-videos/')

In [13]:
# aux.clean()

# Section 2: Downsampling trimmed video

In [14]:
# video_list_s3 = aux.load_s3(bucket = 'leto-dish', prefix = 'original-videos/random-videos/')

In [15]:
# only want the first in the list, the trimmed video
# video_list_s3 = video_list_s3[:-1]

In [16]:
#print the metadata
# print(video_list_s3[0].get_meta_data())

From the metadata we can see that the original codec is h.264, and the video is 1080p

### Downsample the videos

In [17]:
downsampled_video = process.add_label_change_resolution(video_list_s3,"240p")

In [18]:
aux.execute_label_and_write_local(downsampled_video, "modified")

In [19]:
aux.upload_s3(downsampled_video, bucket = 'leto-dish', prefix = 'reduced-videos/ffmpeg-480p/')

In [20]:
downsampled_video[0].get_label()

'-vf scale=426x240:flags=lanczos -c:v libx264 -preset slow -crf 21'

# Section 2: Upsampling trimmed video

In [21]:
video_list_s3 = aux.load_s3(bucket = 'leto-dish', prefix = 'reduced-videos/ffmpeg-480p/')
#print the metadata
print(video_list_s3[0].get_meta_data())

{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'High', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 426, 'height': 240, 'coded_width': 426, 'coded_height': 240, 'closed_captions': 0, 'film_grain': 0, 'has_b_frames': 2, 'pix_fmt': 'yuv420p', 'level': 21, 'color_range': 'tv', 'color_space': 'bt709', 'color_transfer': 'bt709', 'color_primaries': 'bt709', 'chroma_location': 'left', 'field_order': 'progressive', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'id': '0x1', 'r_frame_rate': '25/1', 'avg_frame_rate': '25/1', 'time_base': '1/12800', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 128000, 'duration': '10.000000', 'bit_rate': '23068', 'bits_per_raw_sample': '8', 'nb_frames': '250', 'extradata_size': 48, 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impaired': 0, 'clean_ef

In [22]:
type(video_list_s3[0])

aEye.video.Video

Reading video from S3 using openCV

In [23]:
s3_client = boto3.client('s3')
bucket = 'leto-dish'
key = 'reduced-videos/ffmpeg-480p/resized_demo_10_second_clip.mp4'
url = s3_client.generate_presigned_url( ClientMethod='get_object', Params={ 'Bucket': bucket, 'Key': key } )
video = cv2.VideoCapture(url)

Writing video to S3 using openCV

In [24]:
output_key = 'reconstructed-videos/upsampled_demo.mp4'
output_url = s3_client.generate_presigned_url( ClientMethod='get_object', Params={ 'Bucket': bucket, 'Key': output_key } )

In [ ]:
sr = dnn_superres.DnnSuperResImpl_create()
output_file = 'clip0_comp_decomp.mp4'

# Create a VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*"h264")
out = cv2.VideoWriter(output_file, fourcc, 25.0, (1920, 1080))
dim = (1920, 1080) 
path = "./runner_notebooks/EDSR_x4.pb"
 
sr.readModel(path)
 
sr.setModel("edsr",4)

count = 0

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    result = sr.upsample(frame)

    # Resized image
    resized = cv2.resize(result,dim)

    # Saving the frames with certain names
    cv2.imwrite('./images/%04d.jpg' % count, resized)
    
    # Write the resized frame to the output video file
    out.write(resized)
    
    # Closing the video by Escape button
    if cv2.waitKey(10) == 27:
        break
  
    # Incrementing the variable value by 1
    count += 1

# Release the video capture and writer objects
video.release()
out.release()

# Close all OpenCV windows
cv2.destroyAllWindows()